# Imports

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import json
import re

# Methods

In [3]:
def camel_to_snake(camel_strings):
    snake_strings = []
    
    for string in camel_strings:
        temp = re.sub(r'(?<!^)(?=[A-Z])', '_', string).lower()
        snake_strings.append(temp)
        
    return snake_strings

# Importing Files

In [4]:
with open('Data/Marquee.json', 'r', encoding='utf-8') as f:
    marquee_json = json.load(f)
    
marquee = pd.json_normalize(marquee_json)

marquee.columns = camel_to_snake(marquee.columns)

marquee.head()

,artist_name,segment
0,Cobrah,Previously Active Listeners
1,BLACKPINK,Previously Active Listeners
2,Martin Garrix,Previously Active Listeners
3,Confetti,Previously Active Listeners
4,Demi Lovato,Previously Active Listeners


In [10]:
with open('Data/Playlist1Teixeira.json', 'r', encoding='utf-8') as f:
    playlist_json = json.load(f)

# 2. Normalize as usual
playlist = pd.json_normalize(playlist_json, record_path=['playlists', 'items'], meta=[['playlists', 'name'], ['playlists', 'lastModifiedDate']], sep='_')

playlist.columns = camel_to_snake(playlist.columns) 

playlist.head()

,episode,audiobook,local_track,added_date,track_track_name,track_artist_name,track_album_name,track_track_uri,playlists_name,playlists_last_modified_date
0,None,None,None,2025-11-05,Ferreiro,AYAKASHI,Ferreiro,spotify:track:1bxzMBOWQG6CXwDWPCDQFh,Ordem 🛡️⚜️,2025-12-23
1,None,None,None,2025-11-05,Memórias,AYAKASHI,Memórias,spotify:track:4GJzPRzP8JrVAkF7ckecLL,Ordem 🛡️⚜️,2025-12-23
2,None,None,None,2025-11-05,Natal Macabro (Slashers),Ivou Music,Natal Macabro (Slashers),spotify:track:60QdVpCtnk4AlyiHhIkdya,Ordem 🛡️⚜️,2025-12-23
3,None,None,None,2025-11-05,Manancial,AYAKASHI,Manancial,spotify:track:0UvMzIw4CEVJzQqd440own,Ordem 🛡️⚜️,2025-12-23
4,None,None,None,2025-11-05,Abutre,AYAKASHI,Abutre,spotify:track:6NqsXRUsjFJjJu8BBrJZ10,Ordem 🛡️⚜️,2025-12-23


In [6]:
with open('Data/YourLibrary.json', 'r', encoding='utf-8') as f:
    library_json = json.load(f)

# 2. Normalize as usual
library = pd.json_normalize(library_json, record_path=['tracks'], sep='_')

library.columns = camel_to_snake(library.columns)

library.head()

,artist,album,track,uri
0,Set It Off,Wolf In Sheep's Clothing [REBORN],Wolf In Sheep's Clothing [REBORN],spotify:track:1tpidJ4FBn9TwshePh1bc3
1,Skillet,Awake,Monster,spotify:track:2UREu1Y8CO4jXkbvqAtP7g
2,Thousand Foot Krutch,The End Is Where We Begin,Courtesy Call,spotify:track:0AOmbw8AwDnwXhHC3OhdVB


In [7]:
with open('Data/Streaming_History_Audio_2020-2026.json', 'r', encoding='utf-8') as f:
    history_json = json.load(f)

# 2. Normalize as usual
history = pd.json_normalize(history_json, sep='_')

history.columns = camel_to_snake(history.columns)

history.head()

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2020-10-20T19:36:53Z,"Android OS 10 API 29 (samsung, SM-A307GT)",23600,BR,177.58.181.120,Pretty Savage,BLACKPINK,THE ALBUM,spotify:track:1XnpzbOGptRwfJhZgLbmSr,NaN,...,None,None,None,clickrow,backbtn,False,False,False,NaN,False
1,2020-10-20T19:36:55Z,"Android OS 10 API 29 (samsung, SM-A307GT)",1052,BR,177.58.181.120,How You Like That,BLACKPINK,THE ALBUM,spotify:track:4SFknyjLcyTLJFPKD2m96o,NaN,...,None,None,None,backbtn,backbtn,False,False,False,NaN,False
2,2020-10-20T19:36:56Z,"Android OS 10 API 29 (samsung, SM-A307GT)",0,BR,177.58.181.120,Ice Cream (with Selena Gomez),BLACKPINK,THE ALBUM,spotify:track:4JUPEh2DVSXFGExu4Uxevz,NaN,...,None,None,None,backbtn,backbtn,False,False,False,NaN,False
3,2020-10-20T19:36:56Z,"Android OS 10 API 29 (samsung, SM-A307GT)",0,BR,177.58.181.120,Bet You Wanna (feat. Cardi B),BLACKPINK,THE ALBUM,spotify:track:7iAgNZdotu40NwtoIWJHFe,NaN,...,None,None,None,backbtn,backbtn,False,False,False,NaN,False
4,2020-10-20T19:36:59Z,"Android OS 10 API 29 (samsung, SM-A307GT)",1824,BR,177.58.181.120,Lovesick Girls,BLACKPINK,THE ALBUM,spotify:track:4Ws314Ylb27BVsvlZOy30C,NaN,...,None,None,None,backbtn,fwdbtn,False,False,False,NaN,False


# Creating the main table

In [8]:
# joining history df with marquee df
final_df = pd.merge(history, marquee, how='left', left_on='master_metadata_album_artist_name', right_on='artist_name').drop(columns=['master_metadata_album_artist_name'])

# adding column in_playlist
final_df['in_playlist'] = final_df['master_metadata_track_name'].isin(playlist['track_track_name'])

# adding column in_library
final_df['in_library'] = final_df['master_metadata_track_name'].isin(library['track'])

final_df.head()

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,...,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,artist_name,segment,in_playlist,in_library
0,2020-10-20T19:36:53Z,"Android OS 10 API 29 (samsung, SM-A307GT)",23600,BR,177.58.181.120,Pretty Savage,THE ALBUM,spotify:track:1XnpzbOGptRwfJhZgLbmSr,NaN,NaN,...,backbtn,False,False,False,NaN,False,BLACKPINK,Previously Active Listeners,False,False
1,2020-10-20T19:36:55Z,"Android OS 10 API 29 (samsung, SM-A307GT)",1052,BR,177.58.181.120,How You Like That,THE ALBUM,spotify:track:4SFknyjLcyTLJFPKD2m96o,NaN,NaN,...,backbtn,False,False,False,NaN,False,BLACKPINK,Previously Active Listeners,False,False
2,2020-10-20T19:36:56Z,"Android OS 10 API 29 (samsung, SM-A307GT)",0,BR,177.58.181.120,Ice Cream (with Selena Gomez),THE ALBUM,spotify:track:4JUPEh2DVSXFGExu4Uxevz,NaN,NaN,...,backbtn,False,False,False,NaN,False,BLACKPINK,Previously Active Listeners,False,False
3,2020-10-20T19:36:56Z,"Android OS 10 API 29 (samsung, SM-A307GT)",0,BR,177.58.181.120,Bet You Wanna (feat. Cardi B),THE ALBUM,spotify:track:7iAgNZdotu40NwtoIWJHFe,NaN,NaN,...,backbtn,False,False,False,NaN,False,BLACKPINK,Previously Active Listeners,False,False
4,2020-10-20T19:36:59Z,"Android OS 10 API 29 (samsung, SM-A307GT)",1824,BR,177.58.181.120,Lovesick Girls,THE ALBUM,spotify:track:4Ws314Ylb27BVsvlZOy30C,NaN,NaN,...,fwdbtn,False,False,False,NaN,False,BLACKPINK,Previously Active Listeners,False,False
